# enterpriseknowledge库搭建
> By SherryAGI | Complete的enterprise级 RAG system

## enterpriseknowledge库架构

```
┌─────────────────────────────────────────────────────────────┐
│                    enterpriseknowledge库system                             │
├─────────────────────────────────────────────────────────────┤
│  📥 data源                                                   │
│  ├── Document Repository (Confluence, SharePoint)                        │
│  ├── data库 (MySQL, PostgreSQL)                             │
│  ├── 聊days记录 (Slack, Teams)                                │
│  └── Code库 (GitHub, GitLab)                                │
├─────────────────────────────────────────────────────────────┤
│  🔄 Processing Layer                                                   │
│  ├── Document Parsing → 分chunks → Vectorization                                │
│  └── Incremental Update & Version Control                                     │
├─────────────────────────────────────────────────────────────┤
│  💾 Storage层                                                   │
│  ├── vectordata库 (pgvector / Milvus)                         │
│  └── CNYdataStorage (PostgreSQL)                                │
├─────────────────────────────────────────────────────────────┤
│  🔍 Retrieval层                                                   │
│  ├── 混合Retrieval (vector + keyword)                                │
│  └── Reranker (Reranker)                                      │
├─────────────────────────────────────────────────────────────┤
│  🤖 Generation层                                                   │
│  ├── LLM (GPT-4 / Claude / 本地Models)                        │
│  └── Answer合成 & Source引用                                     │
├─────────────────────────────────────────────────────────────┤
│  🖥️ Application Layer                                                   │
│  ├── Web UI / Chatbot                                       │
│  ├── API API                                               │
│  └── Integration (Slack, Teams, enterprise微信)                          │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
import json
from datetime import datetime
from pathlib import Path
import numpy as np

print('🚀 enterpriseknowledge库system by SherryAGI')

## 1. knowledge库核心类

In [ ]:
class EnterpriseKnowledgeBase:
    """enterpriseknowledge库 by SherryAGI"""
    
    def __init__(self, name="enterpriseknowledge库"):
        self.name = name
        self.documents = []
        self.embeddings = []
        self.categories = set()
        self.stats = {
            "total_docs": 0,
            "total_chunks": 0,
            "last_updated": None
        }
    
    def _embed(self, text):
        """Vectorization（实际使用时替换为真实Models）"""
        np.random.seed(hash(text) % 2**32)
        return np.random.rand(384).tolist()
    
    def add_document(self, content, metadata=None):
        """Add document"""
        doc = {
            "id": len(self.documents),
            "content": content,
            "metadata": metadata or {},
            "created_at": datetime.now().isoformat(),
            "embedding": self._embed(content)
        }
        
        self.documents.append(doc)
        self.embeddings.append(doc["embedding"])
        
        if metadata and "category" in metadata:
            self.categories.add(metadata["category"])
        
        self.stats["total_docs"] += 1
        self.stats["last_updated"] = datetime.now().isoformat()
        
        return doc["id"]
    
    def search(self, query, top_k=5, category=None):
        """搜索knowledge库"""
        from sklearn.metrics.pairwise import cosine_similarity
        
        query_embedding = self._embed(query)
        
        # 计算Similarity
        similarities = cosine_similarity([query_embedding], self.embeddings)[0]
        
        # 过滤category
        results = []
        for i, (sim, doc) in enumerate(zip(similarities, self.documents)):
            if category and doc["metadata"].get("category") != category:
                continue
            results.append((sim, doc))
        
        # 排序
        results.sort(key=lambda x: -x[0])
        
        return [
            {
                "score": float(sim),
                "content": doc["content"],
                "metadata": doc["metadata"],
                "id": doc["id"]
            }
            for sim, doc in results[:top_k]
        ]
    
    def ask(self, question, top_k=3):
        """问答API"""
        # Retrieval相关文档
        results = self.search(question, top_k)
        
        # 构建上下文
        context = "\n\n".join([
            f"[Source: {r['metadata'].get('source', 'Unknown')}]\n{r['content']}"
            for r in results
        ])
        
        # 构建 Prompt
        prompt = f"""基于以下enterpriseknowledge库内容回答Question。请引用Source。

knowledge库内容：
{context}

Question：{question}

Answer："""
        
        return {
            "question": question,
            "prompt": prompt,
            "sources": [r["metadata"].get("source", "Unknown") for r in results],
            "context_length": len(context)
        }
    
    def get_stats(self):
        """Get stats信息"""
        return {
            **self.stats,
            "categories": list(self.categories),
            "category_count": len(self.categories)
        }

print('✅ EnterpriseKnowledgeBase defined')

## 2. 初始化knowledge库

In [ ]:
# 创建knowledge库
kb = EnterpriseKnowledgeBase("SherryAGI enterpriseknowledge库")

# 添加 HR 政策
hr_docs = [
    ("Annual leave政策：after1years entitled to5daysAnnual leave，each additional1year adds1days，maximum15days。Annual leaverequires advance notice of申请，不可跨年累积。", 
     {"source": "HR手册", "category": "HR", "tags": ["Annual leave", "休假"]}),
    ("Sick leave政策：requires hospital certificate，annual cumulative maximum30days。超过3days需HRapproval。",
     {"source": "HR手册", "category": "HR", "tags": ["Sick leave", "休假"]}),
    ("报销标准：Travel expenseseconomy class tickets、3-star hotels；Meal expensesdaily limit100CNY；500CNYrequires manager approval。",
     {"source": "Finance制度", "category": "Finance", "tags": ["报销", "差旅"]}),
]

# 添加 IT 支持
it_docs = [
    ("Password reset：方法1-自助重置（访问 reset.company.com）；方法2-contactIThotline8888。",
     {"source": "IT手册", "category": "IT", "tags": ["密码", "账号"]}),
    ("VPNConnection：下载customer端，使用工号login，选择最近服务器。遇到QuestioncontactIT。",
     {"source": "IT手册", "category": "IT", "tags": ["VPN", "远程"]}),
    ("Equipment request：通过OAsystem提交申请，需部门主管approval，IT部门3个工作日内处理。",
     {"source": "IT手册", "category": "IT", "tags": ["设备", "申请"]}),
]

# 添加产品knowledge
product_docs = [
    ("产品A功能：支持dataanalysis、报表Generation、自动化处理。Use case于中大型enterprise。",
     {"source": "产品文档", "category": "产品", "tags": ["产品A", "功能"]}),
    ("产品A定价：基础版999CNY/月，Professional版2999CNY/月，enterprise版按需定制。",
     {"source": "产品文档", "category": "产品", "tags": ["产品A", "定价"]}),
]

# 批量添加
for content, meta in hr_docs + it_docs + product_docs:
    kb.add_document(content, meta)

print(f'✅ knowledge库初始化完成')
print(json.dumps(kb.get_stats(), indent=2, ensure_ascii=False))

## 3. Test问答

In [ ]:
# TestQuestion
questions = [
    "我有多少daysAnnual leave？",
    "What to do if I forgot my password？",
    "What is the travel reimbursement policy？",
    "产品AHow much does it cost？"
]

for q in questions:
    print(f'\n❓ Question: {q}')
    result = kb.ask(q)
    print(f'📚 Source: {result["sources"]}')
    print(f'📝 Context length: {result["context_length"]} characters')

## 4. 按category搜索

In [ ]:
# Search only IT category
print('🔍 搜索 IT category: "怎么Connection公司网络"\n')
results = kb.search("怎么Connection公司网络", top_k=3, category="IT")

for i, r in enumerate(results):
    print(f'{i+1}. [{r["metadata"]["source"]}] (Similarity: {r["score"]:.3f})')
    print(f'   {r["content"][:60]}...')

## 5. deployment建议

In [ ]:
deployment_guide = """
🚀 enterpriseknowledge库deploymentguide by SherryAGI

1️⃣ Development Environment
   - vector库: Chroma (embedded)
   - LLM: OpenAI API
   - Framework: LangChain

2️⃣ Production Environment (Small scale <10documents)
   - vector库: pgvector (PostgreSQL extension)
   - LLM: Azure OpenAI (Swissnode)
   - deployment: Docker + Kubernetes

3️⃣ Production Environment (Large scale >100documents)
   - vector库: Milvus / Qdrant
   - LLM: 本地deployment (Llama / Qwen)
   - deployment: Distributed cluster

4️⃣ Swisscompliancedeployment
   - data驻留: Azure Switzerland North
   - Encryption: AES-256 静态Encryption
   - Audit: Complete操作日志
   - compliance: FADP, GDPR

5️⃣ recommended开源方案
   - RAGFlow: Professional RAG Engine
   - Dify: 低Code AI Platform
   - AnythingLLM: All-in-one desktop app
"""

print(deployment_guide)

## 6. API APIExample

In [ ]:
# FastAPI APIExample
api_code = '''
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="enterpriseknowledge库 API by SherryAGI")

class Question(BaseModel):
    query: str
    category: str = None
    top_k: int = 3

@app.post("/ask")
async def ask(q: Question):
    """问答API"""
    results = kb.search(q.query, q.top_k, q.category)
    return {
        "question": q.query,
        "results": results
    }

@app.post("/add")
async def add_doc(content: str, metadata: dict = None):
    """Add document"""
    doc_id = kb.add_document(content, metadata)
    return {"id": doc_id, "status": "success"}

@app.get("/stats")
async def get_stats():
    """Get stats"""
    return kb.get_stats()
'''

print('📡 FastAPI APIExample:')
print(api_code)

---
**By SherryAGI** | [DigitalTransformationAI](https://github.com/AIB612/DigitalTransformationAI)